## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
dbutils.fs.ls("/FileStore/tables/")

#dbutils.fs.rm("/FileStore/tables/player_stats/",True)
#dbutils.fs.rm("/FileStore/tables/exp_standings/",True)
#dbutils.fs.rm("/FileStore/tables/mvp_voting/",True)
#dbutils.fs.rm("/FileStore/tables/teams.csv",True)

Out[31]: True

In [0]:
from pyspark.sql.functions import col,when,lit,round,split,monotonically_increasing_id

In [0]:
# File location and type

def load_year(year):
    file_location = "/FileStore/tables/mvp_voting/"
    file_name = f"mvp_{year}.csv"
    file_type = "csv"

    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","

    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(file_location+file_name) \
      .fillna('0.0')

    #display(df)
   
    return df.withColumn("Year",col("Year").cast('int')) \
            .withColumn("Age",col("Age").cast('int')) \
            .withColumn("FirstPlace",col("FirstPlace").cast('float')) \
            .withColumn("PtsWon",col("PtsWon").cast('float')) \
            .withColumn("PtsMax",col("PtsMax").cast('float')) \
            .withColumn("%",col("%").cast('float')) \
            .sort(col("PtsWon").desc()).withColumn("Rank",(monotonically_increasing_id()+1))

In [0]:
result = []
tmp = load_year(1990)
for y in range(1991,2022):
    tmp = tmp.union(load_year(y))
    print("Done with year",y)

result = tmp
#display(result)
#result = result.withColumnRenamed("%","Procent").withColumnRenamed("3P%","ThreePP") \
#            .withColumnRenamed("FG%","FGP").withColumnRenamed("WS/48","WS48")
result.printSchema()

Done with year 1991
Done with year 1992
Done with year 1993
Done with year 1994
Done with year 1995
Done with year 1996
Done with year 1997
Done with year 1998
Done with year 1999
Done with year 2000
Done with year 2001
Done with year 2002
Done with year 2003
Done with year 2004
Done with year 2005
Done with year 2006
Done with year 2007
Done with year 2008
Done with year 2009
Done with year 2010
Done with year 2011
Done with year 2012
Done with year 2013
Done with year 2014
Done with year 2015
Done with year 2016
Done with year 2017
Done with year 2018
Done with year 2019
Done with year 2020
Done with year 2021
root
 |-- Player: string (nullable = false)
 |-- Year: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Team: string (nullable = false)
 |-- FirstPlace: float (nullable = true)
 |-- PtsWon: float (nullable = true)
 |-- PtsMax: float (nullable = true)
 |-- %: float (nullable = true)
 |-- GP: string (nullable = false)
 |-- MP: string (nullable = false)
 |-- PTS:

In [0]:
result = result

#result.printSchema()

In [0]:
result.select(col("Player"),col("Year"),col("PtsWon"),col("Rank")).display()

Player,Year,PtsWon,Rank
Magic Johnson,1990,636.0,1
Charles Barkley,1990,614.0,2
Michael Jordan,1990,564.0,3
Karl Malone,1990,214.0,4
Patrick Ewing,1990,162.0,5
David Robinson,1990,102.0,6
Hakeem Olajuwon,1990,64.0,7
Tom Chambers,1990,12.0,8
John Stockton,1990,9.0,9
Larry Bird,1990,5.0,10


In [0]:
# Create a view or table

temp_table_name = "mvp_csv"

result.createOrReplaceTempView(temp_table_name)

In [0]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.
# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.
# To do so, choose your table name and uncomment the bottom line.

dbutils.fs.rm('dbfs:/user/hive/warehouse/mvp_votes', True)

permanent_table_name = "mvp_votes"

result.write.mode("overwrite").format("parquet").saveAsTable(permanent_table_name)

In [0]:
%sql

/* Query the created temp table in a SQL cell */

select * from `mvp_votes`

Player,Year,Age,Team,FirstPlace,PtsWon,PtsMax,%,GP,MP,PTS,TRP,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Rank
Karl Malone,1999,35,UTA,44.0,827.0,1180.0,0.701,49,37.4,23.8,9.4,4.1,1.3,0.6,.493,.000,.788,9.6,.252,1
Alonzo Mourning,1999,28,MIA,36.0,773.0,1180.0,0.655,46,38.1,20.1,11.0,1.6,0.7,3.9,.511,.000,.652,7.9,.216,2
Tim Duncan,1999,22,SAS,30.0,740.0,1180.0,0.627,50,39.3,21.7,11.4,2.4,0.9,2.5,.495,.143,.690,8.7,.213,3
Allen Iverson,1999,23,PHI,5.0,319.0,1180.0,0.27,48,41.5,26.8,4.9,4.6,2.3,0.1,.412,.291,.751,7.2,.173,4
Jason Kidd,1999,25,PHO,2.0,159.0,1180.0,0.135,50,41.2,16.9,6.8,10.8,2.3,0.4,.444,.366,.757,8.1,.188,5
Shaquille O'Neal,1999,26,LAL,1.0,89.0,1180.0,0.075,49,34.8,26.3,10.7,2.3,0.7,1.7,.576,.000,.540,9.0,.255,6
Chris Webber,1999,25,SAC,0.0,51.0,1180.0,0.043,42,40.9,20.0,13.0,4.1,1.4,2.1,.486,.118,.454,3.6,.102,7
Grant Hill,1999,26,DET,0.0,39.0,1180.0,0.033,50,37.0,21.1,7.1,6.0,1.6,0.5,.479,.000,.752,7.3,.189,8
Gary Payton,1999,30,SEA,0.0,35.0,1180.0,0.03,50,40.2,21.7,4.9,8.7,2.2,0.2,.434,.295,.721,7.2,.172,9
Kevin Garnett,1999,22,MIN,0.0,9.0,1180.0,0.008,47,37.9,20.8,10.4,4.3,1.7,1.8,.460,.286,.704,5.4,.146,10


In [0]:
file_location = "/FileStore/tables/player_stats/"
file_name = "player_stats.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location+file_name) \
  .fillna('0.0')

temp_table_name = "player_stats"

df.createOrReplaceTempView(temp_table_name)

In [0]:
%sql
select * from `player_stats` 

Player,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
Allen Iverson,1999-00,24,PHI,NBA,SG,70,70,40.8,10.4,24.8,.421,1.3,3.7,.341,9.1,21.0,.435,.446,6.3,8.9,.713,1.0,2.8,3.8,4.7,2.1,0.1,3.3,2.3,28.4,20.0,.496,.151,.358,2.7,7.7,5.2,23.0,2.6,0.1,10.3,34.4,3.3,3.6,6.9,.116,3.4,-0.2,3.2,3.8
Kobe Bryant,1999-00,21,LAL,NBA,SG,66,62,38.2,8.4,17.9,.468,0.7,2.2,.319,7.7,15.7,.489,.488,5.0,6.1,.821,1.6,4.7,6.3,4.9,1.6,0.9,2.8,3.3,22.5,21.7,.546,.122,.341,4.6,12.9,8.8,22.4,2.2,1.7,11.8,26.8,6.1,4.5,10.6,.202,3.5,1.6,5.1,4.5
Darrell Armstrong,1999-00,31,ORL,NBA,PG,82,82,31.6,5.9,13.6,.433,1.7,4.9,.340,4.2,8.7,.485,.494,2.7,3.0,.911,0.8,2.5,3.3,6.1,2.1,0.1,3.0,1.7,16.2,19.5,.542,.360,.221,2.7,8.6,5.7,31.4,3.2,0.2,16.8,23.9,4.6,3.4,8.0,.149,3.6,0.9,4.5,4.3
Dikembe Mutombo,1999-00,33,ATL,NBA,C,82,82,36.4,3.9,7.0,.562,0.0,0.0,0.0,3.9,7.0,.562,.562,3.6,5.1,.708,3.7,10.4,14.1,1.3,0.3,3.3,2.1,3.0,11.5,19.4,.621,.000,.735,11.2,31.3,21.2,5.4,0.5,5.9,18.7,13.8,5.6,4.3,9.9,.160,0.9,1.5,2.4,3.3
Gary Payton,1999-00,31,SEA,NBA,PG,82,82,41.8,9.1,20.3,.448,2.2,6.3,.340,7.0,14.0,.497,.502,3.8,5.2,.735,1.2,5.2,6.5,8.9,1.9,0.2,2.7,2.2,24.2,23.6,.535,.312,.254,3.2,13.9,8.5,37.7,2.3,0.4,10.8,26.8,10.5,3.5,13.9,.195,5.8,0.7,6.4,7.3
Tim Duncan,1999-00,23,SAS,NBA,PF,74,74,38.9,8.5,17.3,.490,0.0,0.1,.091,8.5,17.2,.494,.491,6.2,8.1,.761,3.5,8.9,12.4,3.2,0.9,2.2,3.3,2.8,23.2,24.8,.555,.009,.471,10.8,24.8,18.1,15.5,1.2,4.0,13.5,28.7,6.7,6.4,13.0,.218,3.8,1.8,5.6,5.5
Jason Kidd,1999-00,26,PHO,NBA,PG,67,67,39.0,5.2,12.8,.409,0.8,2.5,.337,4.4,10.3,.427,.442,3.0,3.7,.829,1.4,5.8,7.2,10.1,2.0,0.4,3.4,2.2,14.3,18.4,.498,.194,.287,4.2,16.1,10.3,40.1,2.6,0.8,19.0,20.0,3.7,4.8,8.5,.156,2.3,2.0,4.3,4.2
Eddie Jones,1999-00,28,CHH,NBA,SG,72,72,39.0,6.6,15.5,.427,1.8,4.7,.375,4.9,10.8,.450,.484,5.0,5.8,.864,1.1,3.6,4.8,4.2,2.7,0.7,2.2,2.4,20.1,19.9,.555,.305,.374,3.3,10.3,6.9,19.0,3.5,1.2,10.9,23.4,5.7,4.4,10.1,.172,2.9,2.0,4.9,4.9
Bo Outlaw,1999-00,28,ORL,NBA,PF,82,55,28.4,2.5,4.1,.602,0.0,0.0,.000,2.5,4.1,.607,.602,1.0,2.0,.506,2.5,3.9,6.4,3.0,1.4,1.8,1.6,2.5,6.0,15.3,.597,.009,.478,9.5,15.1,12.3,14.7,2.4,4.5,24.5,9.8,2.8,4.0,6.8,.140,0.9,3.6,4.5,3.8
Chris Webber,1999-00,26,SAC,NBA,PF,75,75,38.4,10.0,20.6,.483,0.4,1.3,.284,9.6,19.4,.496,.492,4.1,5.5,.751,2.5,8.0,10.5,4.6,1.6,1.7,2.9,3.5,24.5,23.4,.530,.061,.267,6.8,21.8,14.3,21.1,2.0,2.9,11.2,28.2,5.7,5.0,10.7,.179,3.6,2.1,5.7,5.6


In [0]:
df = sqlContext.sql("select * from `player_stats` ")
#display(df)
df.printSchema()

root
 |-- Player: string (nullable = false)
 |-- Season: string (nullable = false)
 |-- Age: string (nullable = false)
 |-- Tm: string (nullable = false)
 |-- Lg: string (nullable = false)
 |-- Pos: string (nullable = false)
 |-- G: string (nullable = false)
 |-- GS: string (nullable = false)
 |-- MP: string (nullable = false)
 |-- FG: string (nullable = false)
 |-- FGA: string (nullable = false)
 |-- FG%: string (nullable = false)
 |-- 3P: string (nullable = false)
 |-- 3PA: string (nullable = false)
 |-- 3P%: string (nullable = false)
 |-- 2P: string (nullable = false)
 |-- 2PA: string (nullable = false)
 |-- 2P%: string (nullable = false)
 |-- eFG%: string (nullable = false)
 |-- FT: string (nullable = false)
 |-- FTA: string (nullable = false)
 |-- FT%: string (nullable = false)
 |-- ORB: string (nullable = false)
 |-- DRB: string (nullable = false)
 |-- TRB: string (nullable = false)
 |-- AST: string (nullable = false)
 |-- STL: string (nullable = false)
 |-- BLK: string (nullable

In [0]:
schema = df.schema
print(len(schema))
print(schema[8].name)
#df.withColumn("Season",col("Season").cast('int')).printSchema()
tmp = df

for i in range(8,51):
    name = schema[i].name
    tmp = tmp.withColumn(name,col(name).cast('float'))

tmp.printSchema()
df = tmp

51
MP
root
 |-- Player: string (nullable = false)
 |-- Season: string (nullable = false)
 |-- Age: string (nullable = false)
 |-- Tm: string (nullable = false)
 |-- Lg: string (nullable = false)
 |-- Pos: string (nullable = false)
 |-- G: string (nullable = false)
 |-- GS: string (nullable = false)
 |-- MP: float (nullable = true)
 |-- FG: float (nullable = true)
 |-- FGA: float (nullable = true)
 |-- FG%: float (nullable = true)
 |-- 3P: float (nullable = true)
 |-- 3PA: float (nullable = true)
 |-- 3P%: float (nullable = true)
 |-- 2P: float (nullable = true)
 |-- 2PA: float (nullable = true)
 |-- 2P%: float (nullable = true)
 |-- eFG%: float (nullable = true)
 |-- FT: float (nullable = true)
 |-- FTA: float (nullable = true)
 |-- FT%: float (nullable = true)
 |-- ORB: float (nullable = true)
 |-- DRB: float (nullable = true)
 |-- TRB: float (nullable = true)
 |-- AST: float (nullable = true)
 |-- STL: float (nullable = true)
 |-- BLK: float (nullable = true)
 |-- TOV: float (nullabl

In [0]:
df = df.withColumn("Season",(df.Season.cast("String")[0:4]).cast("int")+1).withColumn("Age",col("Age").cast("int"))
df = df.withColumn("Pos",when((df.Pos.cast("String")[0:3] == "PG") | (df.Pos.cast("String")[0:3] == "SG"),"G") \
             .when((df.Pos.cast("String")[0:3] == "SF") | (df.Pos.cast("String")[0:3] == "PF"),"F") \
             .otherwise("C"))


display(df)

Player,Season,Age,Tm,Lg,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
Allen Iverson,2000,24,PHI,NBA,G,70,70,40.8,10.4,24.8,0.421,1.3,3.7,0.341,9.1,21.0,0.435,0.446,6.3,8.9,0.713,1.0,2.8,3.8,4.7,2.1,0.1,3.3,2.3,28.4,20.0,0.496,0.151,0.358,2.7,7.7,5.2,23.0,2.6,0.1,10.3,34.4,3.3,3.6,6.9,0.116,3.4,-0.2,3.2,3.8
Kobe Bryant,2000,21,LAL,NBA,G,66,62,38.2,8.4,17.9,0.468,0.7,2.2,0.319,7.7,15.7,0.489,0.488,5.0,6.1,0.821,1.6,4.7,6.3,4.9,1.6,0.9,2.8,3.3,22.5,21.7,0.546,0.122,0.341,4.6,12.9,8.8,22.4,2.2,1.7,11.8,26.8,6.1,4.5,10.6,0.202,3.5,1.6,5.1,4.5
Darrell Armstrong,2000,31,ORL,NBA,G,82,82,31.6,5.9,13.6,0.433,1.7,4.9,0.34,4.2,8.7,0.485,0.494,2.7,3.0,0.911,0.8,2.5,3.3,6.1,2.1,0.1,3.0,1.7,16.2,19.5,0.542,0.36,0.221,2.7,8.6,5.7,31.4,3.2,0.2,16.8,23.9,4.6,3.4,8.0,0.149,3.6,0.9,4.5,4.3
Dikembe Mutombo,2000,33,ATL,NBA,C,82,82,36.4,3.9,7.0,0.562,0.0,0.0,0.0,3.9,7.0,0.562,0.562,3.6,5.1,0.708,3.7,10.4,14.1,1.3,0.3,3.3,2.1,3.0,11.5,19.4,0.621,0.0,0.735,11.2,31.3,21.2,5.4,0.5,5.9,18.7,13.8,5.6,4.3,9.9,0.16,0.9,1.5,2.4,3.3
Gary Payton,2000,31,SEA,NBA,G,82,82,41.8,9.1,20.3,0.448,2.2,6.3,0.34,7.0,14.0,0.497,0.502,3.8,5.2,0.735,1.2,5.2,6.5,8.9,1.9,0.2,2.7,2.2,24.2,23.6,0.535,0.312,0.254,3.2,13.9,8.5,37.7,2.3,0.4,10.8,26.8,10.5,3.5,13.9,0.195,5.8,0.7,6.4,7.3
Tim Duncan,2000,23,SAS,NBA,F,74,74,38.9,8.5,17.3,0.49,0.0,0.1,0.091,8.5,17.2,0.494,0.491,6.2,8.1,0.761,3.5,8.9,12.4,3.2,0.9,2.2,3.3,2.8,23.2,24.8,0.555,0.009,0.471,10.8,24.8,18.1,15.5,1.2,4.0,13.5,28.7,6.7,6.4,13.0,0.218,3.8,1.8,5.6,5.5
Jason Kidd,2000,26,PHO,NBA,G,67,67,39.0,5.2,12.8,0.409,0.8,2.5,0.337,4.4,10.3,0.427,0.442,3.0,3.7,0.829,1.4,5.8,7.2,10.1,2.0,0.4,3.4,2.2,14.3,18.4,0.498,0.194,0.287,4.2,16.1,10.3,40.1,2.6,0.8,19.0,20.0,3.7,4.8,8.5,0.156,2.3,2.0,4.3,4.2
Eddie Jones,2000,28,CHH,NBA,G,72,72,39.0,6.6,15.5,0.427,1.8,4.7,0.375,4.9,10.8,0.45,0.484,5.0,5.8,0.864,1.1,3.6,4.8,4.2,2.7,0.7,2.2,2.4,20.1,19.9,0.555,0.305,0.374,3.3,10.3,6.9,19.0,3.5,1.2,10.9,23.4,5.7,4.4,10.1,0.172,2.9,2.0,4.9,4.9
Bo Outlaw,2000,28,ORL,NBA,F,82,55,28.4,2.5,4.1,0.602,0.0,0.0,0.0,2.5,4.1,0.607,0.602,1.0,2.0,0.506,2.5,3.9,6.4,3.0,1.4,1.8,1.6,2.5,6.0,15.3,0.597,0.009,0.478,9.5,15.1,12.3,14.7,2.4,4.5,24.5,9.8,2.8,4.0,6.8,0.14,0.9,3.6,4.5,3.8
Chris Webber,2000,26,SAC,NBA,F,75,75,38.4,10.0,20.6,0.483,0.4,1.3,0.284,9.6,19.4,0.496,0.492,4.1,5.5,0.751,2.5,8.0,10.5,4.6,1.6,1.7,2.9,3.5,24.5,23.4,0.53,0.061,0.267,6.8,21.8,14.3,21.1,2.0,2.9,11.2,28.2,5.7,5.0,10.7,0.179,3.6,2.1,5.7,5.6


In [0]:
tmp_stats = df.drop(df.Lg).drop(df.Tm)#.drop(df.Pos)

result = sqlContext.sql("select * from `mvp_votes` ")
tmp_mvp = result.select(col("Player"),col("Year"),col("Age"),col("Team"),col("Rank"),col("FirstPlace"),col("PtsWon"),col("PtsMax"),col("%"))

full_stats = tmp_mvp.join(tmp_stats, ["Player","Age"],"left")
full_stats = full_stats.drop(full_stats.Season)
full_stats.printSchema()

root
 |-- Player: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Team: string (nullable = true)
 |-- Rank: long (nullable = true)
 |-- FirstPlace: float (nullable = true)
 |-- PtsWon: float (nullable = true)
 |-- PtsMax: float (nullable = true)
 |-- %: float (nullable = true)
 |-- Pos: string (nullable = true)
 |-- G: string (nullable = true)
 |-- GS: string (nullable = true)
 |-- MP: float (nullable = true)
 |-- FG: float (nullable = true)
 |-- FGA: float (nullable = true)
 |-- FG%: float (nullable = true)
 |-- 3P: float (nullable = true)
 |-- 3PA: float (nullable = true)
 |-- 3P%: float (nullable = true)
 |-- 2P: float (nullable = true)
 |-- 2PA: float (nullable = true)
 |-- 2P%: float (nullable = true)
 |-- eFG%: float (nullable = true)
 |-- FT: float (nullable = true)
 |-- FTA: float (nullable = true)
 |-- FT%: float (nullable = true)
 |-- ORB: float (nullable = true)
 |-- DRB: float (nullable = true)
 |-- TRB: float (nullable

In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/full_mvp_stats', True)

permanent_table_name = "full_mvp_stats"

full_stats.write.mode("overwrite").format("parquet").saveAsTable(permanent_table_name)

In [0]:
%sql

select * from `full_mvp_stats` 

Player,Age,Year,Team,Rank,FirstPlace,PtsWon,PtsMax,%,Pos,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
Karl Malone,35,1999,UTA,1,44.0,827.0,1180.0,0.701,F,49,49,37.4,8.0,16.3,0.493,0.0,0.0,0.0,8.0,16.2,0.494,0.493,7.7,9.8,0.788,2.2,7.3,9.4,4.1,1.3,0.6,3.3,2.7,23.8,25.6,0.577,0.001,0.602,7.7,22.1,15.4,23.0,1.9,1.2,13.8,30.5,6.2,3.4,9.6,0.252,5.3,1.4,6.7,4.0
Alonzo Mourning,28,1999,MIA,2,36.0,773.0,1180.0,0.655,C,46,46,38.1,7.0,13.8,0.511,0.0,0.0,0.0,7.0,13.7,0.513,0.511,6.0,9.2,0.652,3.6,7.4,11.0,1.6,0.7,3.9,3.0,3.5,20.1,24.6,0.563,0.003,0.667,12.3,22.0,17.5,8.7,1.1,7.7,14.5,27.1,4.0,3.9,7.9,0.216,2.5,2.2,4.7,3.0
Tim Duncan,22,1999,SAS,3,30.0,740.0,1180.0,0.627,F,50,50,39.3,8.4,16.9,0.495,0.0,0.1,0.143,8.3,16.8,0.498,0.495,4.9,7.2,0.69,3.2,8.2,11.4,2.4,0.9,2.5,2.9,2.9,21.7,23.2,0.541,0.008,0.424,9.7,22.2,16.3,12.1,1.2,4.4,12.7,27.2,4.0,4.7,8.7,0.213,2.7,1.8,4.5,3.2
Allen Iverson,23,1999,PHI,4,5.0,319.0,1180.0,0.27,G,48,48,41.5,9.1,22.0,0.412,1.2,4.1,0.291,7.9,17.9,0.44,0.439,7.4,9.9,0.751,1.4,3.5,4.9,4.6,2.3,0.1,3.5,2.0,26.8,22.2,0.508,0.188,0.449,3.8,10.1,6.9,24.2,3.0,0.3,11.7,32.6,4.3,2.9,7.2,0.173,5.1,0.6,5.7,3.9
Jason Kidd,25,1999,PHO,5,2.0,159.0,1180.0,0.135,G,50,50,41.2,6.2,14.0,0.444,0.9,2.5,0.366,5.3,11.5,0.461,0.476,3.6,4.8,0.757,1.7,5.0,6.8,10.8,2.3,0.4,3.0,2.2,16.9,22.5,0.527,0.176,0.342,4.8,14.5,9.6,44.0,3.0,0.7,15.7,21.4,5.8,2.3,8.1,0.188,4.4,1.1,5.5,3.9
Shaquille O'Neal,26,1999,LAL,6,1.0,89.0,1180.0,0.075,C,49,49,34.8,10.4,18.1,0.576,0.0,0.0,0.0,10.4,18.0,0.577,0.576,5.5,10.2,0.54,3.8,6.9,10.7,2.3,0.7,1.7,2.5,3.2,26.3,30.6,0.584,0.001,0.563,13.1,22.6,18.0,14.4,1.1,3.5,10.0,32.4,7.2,1.9,9.0,0.255,7.5,-0.4,7.1,3.9
Chris Webber,25,1999,SAC,7,0.0,51.0,1180.0,0.043,F,42,42,40.9,9.0,18.5,0.486,0.1,0.8,0.118,8.9,17.7,0.503,0.488,1.9,4.1,0.454,3.5,9.4,13.0,4.1,1.4,2.1,3.5,3.5,20.0,19.5,0.491,0.044,0.224,9.2,24.2,16.7,17.8,1.7,3.3,14.8,24.9,1.0,2.7,3.6,0.102,0.8,1.1,1.9,1.7
Grant Hill,26,1999,DET,8,0.0,39.0,1180.0,0.033,F,50,50,37.0,7.7,16.0,0.479,0.0,0.3,0.0,7.7,15.8,0.487,0.479,5.7,7.6,0.752,1.3,5.8,7.1,6.0,1.6,0.5,3.7,2.3,21.1,23.9,0.543,0.017,0.473,4.3,19.3,11.7,33.6,2.4,1.2,16.0,29.6,4.5,2.8,7.3,0.189,4.8,1.4,6.2,3.8
Gary Payton,30,1999,SEA,9,0.0,35.0,1180.0,0.03,G,50,50,40.2,8.0,18.5,0.434,1.7,5.6,0.295,6.4,12.8,0.495,0.479,4.0,5.5,0.721,1.2,3.6,4.9,8.7,2.2,0.2,3.1,2.3,21.7,23.1,0.519,0.304,0.299,3.5,10.6,7.0,41.3,2.9,0.4,12.9,27.1,6.0,1.2,7.2,0.172,5.1,0.0,5.1,3.6
Kevin Garnett,22,1999,MIN,10,0.0,9.0,1180.0,0.008,F,47,47,37.9,8.8,19.1,0.46,0.1,0.3,0.286,8.7,18.9,0.463,0.462,3.1,4.4,0.704,3.5,6.9,10.4,4.3,1.7,1.8,2.9,3.2,20.8,22.4,0.493,0.016,0.229,9.7,22.0,15.4,21.3,2.3,3.5,12.0,27.8,2.4,3.0,5.4,0.146,3.6,1.7,5.3,3.3


In [0]:
def load_year_stand(year):
    file_location = "/FileStore/tables/exp_standings/"
    file_name = f"standings_{year}.csv"
    file_type = "csv"

    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","

    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(file_location+file_name) \
      .fillna('0.0')
    
    df = df.withColumn("Year",lit(year))
    #display(df)
   
    return df

In [0]:
standings = []
tmp = load_year_stand(1990)
for y in range(1991,2022):
    tmp = tmp.union(load_year_stand(y))
    print("Done with year",y)

standings_first = tmp
display(standings_first)

#standings.printSchema()

Done with year 1991
Done with year 1992
Done with year 1993
Done with year 1994
Done with year 1995
Done with year 1996
Done with year 1997
Done with year 1998
Done with year 1999
Done with year 2000
Done with year 2001
Done with year 2002
Done with year 2003
Done with year 2004
Done with year 2005
Done with year 2006
Done with year 2007
Done with year 2008
Done with year 2009
Done with year 2010
Done with year 2011
Done with year 2012
Done with year 2013
Done with year 2014
Done with year 2015
Done with year 2016
Done with year 2017
Done with year 2018
Done with year 2019
Done with year 2020
Done with year 2021


Rk,Team,Overall,Home,Road,E,W,A,C,SE,NW,P,SW,Pre,Post,≤3,≥10,Year
1,Los Angeles Lakers,63-19,37-4,26-15,19-7,44-12,10-2,9-5,22-6,22-6,35-11,28-8,6-7,30-7,11-2,10-4,1990
2,Detroit Pistons,59-23,35-6,24-17,40-14,19-9,18-6,22-8,10-4,9-5,35-14,24-9,6-4,32-8,9-5,9-6,1990
3,Portland Trail Blazers,59-23,35-6,24-17,18-8,41-15,10-2,8-6,21-7,20-8,33-13,26-10,7-6,33-8,11-3,8-7,1990
4,San Antonio Spurs,56-26,34-7,22-19,16-10,40-16,9-3,7-7,19-9,21-7,32-14,24-12,10-3,23-9,8-5,11-2,1990
5,Chicago Bulls,55-27,36-5,19-22,37-17,18-10,17-7,20-10,9-5,9-5,28-19,27-8,8-7,26-12,8-6,11-3,1990
6,Utah Jazz,55-27,36-5,19-22,17-9,38-18,8-4,9-5,21-7,17-11,33-14,22-13,8-3,35-15,8-4,11-5,1990
7,Phoenix Suns,54-28,32-9,22-19,16-10,38-18,9-3,7-7,18-10,20-8,28-17,26-11,6-5,38-7,5-6,7-7,1990
8,Philadelphia 76ers,53-29,34-7,19-22,37-17,16-12,19-7,18-10,8-6,8-6,30-18,23-11,6-6,30-12,8-4,7-9,1990
9,Boston Celtics,52-30,30-11,22-19,36-18,16-12,19-7,17-11,9-5,7-7,28-18,24-12,7-4,30-11,8-7,9-4,1990
10,Dallas Mavericks,47-35,30-11,17-24,18-8,29-27,8-4,10-4,17-11,12-16,26-22,21-13,7-3,20-19,6-7,9-6,1990


In [0]:
standings = standings_first.select(col("Rk"),col("Team"),col("Year"),col("Overall"),col("Pre"),col("Post"))
standings = standings.withColumn("Wins",col("Overall").cast("String")[0:2].cast("int")) \
         .withColumn("Losses",col("Overall").cast("String")[4:5].cast("int")) \
         .withColumn("GamesNo",col("Wins")+col("Losses")) \
         .withColumn("Win%",round((col("Wins")*1.0)/col("GamesNo"),3))

split_col = split(standings.Pre,'-')
standings = standings.withColumn("WinsPre",split_col.getItem(0).cast("int")) \
            .withColumn("LosesPre",split_col.getItem(1).cast("int")) \
            .withColumn("TotalPre",col("WinsPre")+col("LosesPre")) \
            .withColumn("Win%Pre",round((col("WinsPre")*1.0)/col("TotalPre"),3))

split_col = split(standings.Post,'-')
standings = standings.withColumn("WinsPost",split_col.getItem(0).cast("int")) \
            .withColumn("LosesPost",split_col.getItem(1).cast("int")) \
            .withColumn("TotalPost",col("WinsPost")+col("LosesPost")) \
            .withColumn("Win%Post",round((col("WinsPost")*1.0)/col("TotalPost"),3))

standings.printSchema()
standings = standings.drop(col("Overall")).drop(col("Pre")) \
             .drop(col("Post")).drop(col("WinsPre")).drop(col("LosesPre")) \
             .drop(col("WinsPost")).drop(col("LosesPost"))

root
 |-- Rk: string (nullable = false)
 |-- Team: string (nullable = false)
 |-- Year: integer (nullable = false)
 |-- Overall: string (nullable = false)
 |-- Pre: string (nullable = false)
 |-- Post: string (nullable = false)
 |-- Wins: integer (nullable = true)
 |-- Losses: integer (nullable = true)
 |-- GamesNo: integer (nullable = true)
 |-- Win%: double (nullable = true)
 |-- WinsPre: integer (nullable = true)
 |-- LosesPre: integer (nullable = true)
 |-- TotalPre: integer (nullable = true)
 |-- Win%Pre: double (nullable = true)
 |-- WinsPost: integer (nullable = true)
 |-- LosesPost: integer (nullable = true)
 |-- TotalPost: integer (nullable = true)
 |-- Win%Post: double (nullable = true)



In [0]:
standings.display()

Rk,Team,Year,Wins,Losses,GamesNo,Win%,TotalPre,Win%Pre,TotalPost,Win%Post
1,Los Angeles Lakers,1990,63,19,82,0.768,13,0.462,37,0.811
2,Detroit Pistons,1990,59,23,82,0.72,10,0.6,40,0.8
3,Portland Trail Blazers,1990,59,23,82,0.72,13,0.538,41,0.805
4,San Antonio Spurs,1990,56,26,82,0.683,13,0.769,32,0.719
5,Chicago Bulls,1990,55,27,82,0.671,15,0.533,38,0.684
6,Utah Jazz,1990,55,27,82,0.671,11,0.727,50,0.7
7,Phoenix Suns,1990,54,28,82,0.659,11,0.545,45,0.844
8,Philadelphia 76ers,1990,53,29,82,0.646,12,0.5,42,0.714
9,Boston Celtics,1990,52,30,82,0.634,11,0.636,41,0.732
10,Dallas Mavericks,1990,47,35,82,0.573,10,0.7,39,0.513


In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/team_standings', True)

permanent_table_name = "team_standings"

standings.write.mode("overwrite").format("parquet").saveAsTable(permanent_table_name)

In [0]:
def get_mini():
    file_location = "/FileStore/tables/teams.csv"
    file_type = "csv"

    # CSV options
    infer_schema = "false"
    first_row_is_header = "true"
    delimiter = ","

    # The applied options are for CSV files. For other file types, these will be ignored.
    df = spark.read.format(file_type) \
      .option("inferSchema", infer_schema) \
      .option("header", first_row_is_header) \
      .option("sep", delimiter) \
      .load(file_location)

    #display(df)
    return df

In [0]:
teams_mini = get_mini()
teams_mini.printSchema()
teams_mini = teams_mini.withColumnRenamed("Team","Nickname").withColumnRenamed("TeamName","Team")

standings = teams_mini.join(standings, ["Team"],"left")
standings.display()

root
 |-- Team: string (nullable = true)
 |-- TeamName: string (nullable = true)



Team,Nickname,Rk,Year,Wins,Losses,GamesNo,Win%,TotalPre,Win%Pre,TotalPost,Win%Post
Utah Jazz,UTA,1,2021,52,20,72,0.722,36,0.75,36,0.694
Utah Jazz,UTA,11,2020,44,28,72,0.611,54,0.667,18,0.444
Utah Jazz,UTA,8,2019,50,32,82,0.61,57,0.561,25,0.72
Utah Jazz,UTA,11,2018,48,34,82,0.585,58,0.517,24,0.75
Utah Jazz,UTA,8,2017,51,31,82,0.622,57,0.614,25,0.64
Utah Jazz,UTA,19,2016,40,42,82,0.488,52,0.5,30,0.467
Utah Jazz,UTA,20,2015,38,44,82,0.463,53,0.358,29,0.655
Utah Jazz,UTA,27,2014,25,57,82,0.305,52,0.365,30,0.2
Utah Jazz,UTA,15,2013,43,39,82,0.524,54,0.556,28,0.464
Utah Jazz,UTA,15,2012,36,30,66,0.545,32,0.469,34,0.618


In [0]:
full_players = full_stats.withColumnRenamed("Team","Nickname").join(standings, ["Nickname","Year"], "leftouter") \
                            .drop("Team").withColumnRenamed("Nickname","Team")
full_players = full_players.withColumn("Rk",col("Rk").cast("int")).withColumn("G",col("G").cast("int")).withColumn("Gs",col("Gs").cast("int"))

full_players = full_players.withColumn("VoteType", when(full_players.Rank==1, "Winners") \
                 .when((full_players.Rank<=3) & (full_players.Rank>1),"Finalist") \
                 .when((full_players.Rank<=5) & (full_players.Rank>3),"Contenders") \
                 .otherwise("Other"))

full_players.printSchema()
full_players.display()

root
 |-- Team: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Rank: long (nullable = true)
 |-- FirstPlace: float (nullable = true)
 |-- PtsWon: float (nullable = true)
 |-- PtsMax: float (nullable = true)
 |-- %: float (nullable = true)
 |-- Pos: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- Gs: integer (nullable = true)
 |-- MP: float (nullable = true)
 |-- FG: float (nullable = true)
 |-- FGA: float (nullable = true)
 |-- FG%: float (nullable = true)
 |-- 3P: float (nullable = true)
 |-- 3PA: float (nullable = true)
 |-- 3P%: float (nullable = true)
 |-- 2P: float (nullable = true)
 |-- 2PA: float (nullable = true)
 |-- 2P%: float (nullable = true)
 |-- eFG%: float (nullable = true)
 |-- FT: float (nullable = true)
 |-- FTA: float (nullable = true)
 |-- FT%: float (nullable = true)
 |-- ORB: float (nullable = true)
 |-- DRB: float (nullable = true)
 |-- TRB: float (nullab

Team,Year,Player,Age,Rank,FirstPlace,PtsWon,PtsMax,%,Pos,G,Gs,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Rk,Wins,Losses,GamesNo,Win%,TotalPre,Win%Pre,TotalPost,Win%Post,VoteType
UTA,1999,Karl Malone,35,1,44.0,827.0,1180.0,0.701,F,49,49,37.4,8.0,16.3,0.493,0.0,0.0,0.0,8.0,16.2,0.494,0.493,7.7,9.8,0.788,2.2,7.3,9.4,4.1,1.3,0.6,3.3,2.7,23.8,25.6,0.577,0.001,0.602,7.7,22.1,15.4,23.0,1.9,1.2,13.8,30.5,6.2,3.4,9.6,0.252,5.3,1.4,6.7,4.0,2,37,13,50,0.74,13,0.846,16,0.625,Winners
MIA,1999,Alonzo Mourning,28,2,36.0,773.0,1180.0,0.655,C,46,46,38.1,7.0,13.8,0.511,0.0,0.0,0.0,7.0,13.7,0.513,0.511,6.0,9.2,0.652,3.6,7.4,11.0,1.6,0.7,3.9,3.0,3.5,20.1,24.6,0.563,0.003,0.667,12.3,22.0,17.5,8.7,1.1,7.7,14.5,27.1,4.0,3.9,7.9,0.216,2.5,2.2,4.7,3.0,5,33,17,50,0.66,13,0.692,15,0.667,Finalist
SAS,1999,Tim Duncan,22,3,30.0,740.0,1180.0,0.627,F,50,50,39.3,8.4,16.9,0.495,0.0,0.1,0.143,8.3,16.8,0.498,0.495,4.9,7.2,0.69,3.2,8.2,11.4,2.4,0.9,2.5,2.9,2.9,21.7,23.2,0.541,0.008,0.424,9.7,22.2,16.3,12.1,1.2,4.4,12.7,27.2,4.0,4.7,8.7,0.213,2.7,1.8,4.5,3.2,1,37,13,50,0.74,14,0.429,16,0.875,Finalist
PHI,1999,Allen Iverson,23,4,5.0,319.0,1180.0,0.27,G,48,48,41.5,9.1,22.0,0.412,1.2,4.1,0.291,7.9,17.9,0.44,0.439,7.4,9.9,0.751,1.4,3.5,4.9,4.6,2.3,0.1,3.5,2.0,26.8,22.2,0.508,0.188,0.449,3.8,10.1,6.9,24.2,3.0,0.3,11.7,32.6,4.3,2.9,7.2,0.173,5.1,0.6,5.7,3.9,12,28,22,50,0.56,13,0.615,16,0.438,Contenders
PHO,1999,Jason Kidd,25,5,2.0,159.0,1180.0,0.135,G,50,50,41.2,6.2,14.0,0.444,0.9,2.5,0.366,5.3,11.5,0.461,0.476,3.6,4.8,0.757,1.7,5.0,6.8,10.8,2.3,0.4,3.0,2.2,16.9,22.5,0.527,0.176,0.342,4.8,14.5,9.6,44.0,3.0,0.7,15.7,21.4,5.8,2.3,8.1,0.188,4.4,1.1,5.5,3.9,14,27,23,50,0.54,13,0.615,18,0.389,Contenders
LAL,1999,Shaquille O'Neal,26,6,1.0,89.0,1180.0,0.075,C,49,49,34.8,10.4,18.1,0.576,0.0,0.0,0.0,10.4,18.0,0.577,0.576,5.5,10.2,0.54,3.8,6.9,10.7,2.3,0.7,1.7,2.5,3.2,26.3,30.6,0.584,0.001,0.563,13.1,22.6,18.0,14.4,1.1,3.5,10.0,32.4,7.2,1.9,9.0,0.255,7.5,-0.4,7.1,3.9,9,31,19,50,0.62,15,0.6,17,0.706,Other
SAC,1999,Chris Webber,25,7,0.0,51.0,1180.0,0.043,F,42,42,40.9,9.0,18.5,0.486,0.1,0.8,0.118,8.9,17.7,0.503,0.488,1.9,4.1,0.454,3.5,9.4,13.0,4.1,1.4,2.1,3.5,3.5,20.0,19.5,0.491,0.044,0.224,9.2,24.2,16.7,17.8,1.7,3.3,14.8,24.9,1.0,2.7,3.6,0.102,0.8,1.1,1.9,1.7,15,27,23,50,0.54,13,0.462,19,0.421,Other
DET,1999,Grant Hill,26,8,0.0,39.0,1180.0,0.033,F,50,50,37.0,7.7,16.0,0.479,0.0,0.3,0.0,7.7,15.8,0.487,0.479,5.7,7.6,0.752,1.3,5.8,7.1,6.0,1.6,0.5,3.7,2.3,21.1,23.9,0.543,0.017,0.473,4.3,19.3,11.7,33.6,2.4,1.2,16.0,29.6,4.5,2.8,7.3,0.189,4.8,1.4,6.2,3.8,10,29,21,50,0.58,14,0.571,17,0.647,Other
SEA,1999,Gary Payton,30,9,0.0,35.0,1180.0,0.03,G,50,50,40.2,8.0,18.5,0.434,1.7,5.6,0.295,6.4,12.8,0.495,0.479,4.0,5.5,0.721,1.2,3.6,4.9,8.7,2.2,0.2,3.1,2.3,21.7,23.1,0.519,0.304,0.299,3.5,10.6,7.0,41.3,2.9,0.4,12.9,27.1,6.0,1.2,7.2,0.172,5.1,0.0,5.1,3.6,18,25,25,50,0.5,12,0.667,17,0.412,Other
MIN,1999,Kevin Garnett,22,10,0.0,9.0,1180.0,0.008,F,47,47,37.9,8.8,19.1,0.46,0.1,0.3,0.286,8.7,18.9,0.463,0.462,3.1,4.4,0.704,3.5,6.9,10.4,4.3,1.7,1.8,2.9,3.2,20.8,22.4,0.493,0.016,0.229,9.7,22.0,15.4,21.3,2.3,3.5,12.0,27.8,2.4,3.0,5.4,0.146,3.6,1.7,5.3,3.3,17,25,25,50,0.5,13,0.615,17,0.529,Other


In [0]:
dbutils.fs.rm('dbfs:/user/hive/warehouse/full_player_stats_with_teams', True)

permanent_table_name = "full_player_stats_with_teams"

full_players.write.mode("overwrite").format("parquet").saveAsTable(permanent_table_name)